# Use the style transformation network to do a video transformation. 

download the style image.

In [ ]:
!wget http://107.172.22.247/NST/impression_sunrise.jpg

!wget http://107.172.22.247/NST/3.jpg

!mkdir checkpoint
!mkdir model

--2020-11-30 21:58:40--  http://107.172.22.247/NST/impression_sunrise.jpg
Connecting to 107.172.22.247:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29745 (29K) [image/jpeg]
Saving to: ‘impression_sunrise.jpg’

impression_sunrise. 100%[===================>]  29.05K  --.-KB/s    in 0.03s   

2020-11-30 21:58:40 (851 KB/s) - ‘impression_sunrise.jpg’ saved [29745/29745]

--2020-11-30 21:58:40--  http://107.172.22.247/NST/3.jpg
Connecting to 107.172.22.247:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15606 (15K) [image/jpeg]
Saving to: ‘3.jpg’

3.jpg               100%[===================>]  15.24K  --.-KB/s    in 0.03s   

2020-11-30 21:58:40 (454 KB/s) - ‘3.jpg’ saved [15606/15606]



build the model 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Nov 27

@author: Ganyu Wang

Train Draft 4

use the dataset intel recognition. 

train a identify transformation model. 

save model, reconstructruct model.

Write the custom loss function again. 
    the custom loss function should return the 

Predict.
Visualization. 

"""

import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt


#%% build models

def residual_block(y, nb_channels, _strides=(1, 1), _project_shortcut=False):
    shortcut = y

    # down-sampling is performed with a stride of 2
    y = layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    y = layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
    y = layers.BatchNormalization()(y)

    # identity shortcuts used directly when the input and output are of the same dimensions
    if _project_shortcut or _strides != (1, 1):
        # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
        # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
        shortcut = layers.Conv2D(nb_channels, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    y = layers.add([shortcut, y])
    y = layers.LeakyReLU()(y)

    return y

def Identical_transform_model(y):
    """
    transform_model, channel last, 
        input  (batch_size, width, height, channel) -> (BS, 256, 256, 3)
        output (batch_size, width, height, channel) -> (BS, 256, 256, 3)
    
    """
    d = Conv2D(32, (9, 9), input_shape=(256, 256, 3), padding='same', activation='relu')(y)
    d = Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(d)
    d = Conv2D(128, (3, 3), activation='relu', padding='same', strides=2)(d)
    d = residual_block(d, 128)
    d = residual_block(d, 128)
    d = residual_block(d, 128)
    d = residual_block(d, 128)
    d = residual_block(d, 128)
    d = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=2)(d)
    d = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=2)(d)
    d = Conv2DTranspose(3, (9, 9), activation='relu', padding='same', strides=1)(d)
    
    return d

inputs = keras.Input(shape=(256, 256, 3))
outputs = Identical_transform_model(inputs)
transform_model = keras.Model(inputs=inputs, outputs=outputs)



80142336/80134624 [==============================] - 1s 0us/step


** Upload the weight h5 file here ** 

It takes 1 minutes in google colab. 


In [ ]:
transform_model.load_weights("model_draft_6.h5")


In [ ]:
#%% predict 

test_img = load_image("3.jpg")
pred_img = transform_model.predict(test_img)


plt.figure()
plt.imshow(test_img[0,])

plt.figure()
plt.imshow(pred_img[0,])


Download video.

In [ ]:
!wget http://107.172.22.247/NST/video.mp4


--2020-11-30 22:02:09--  http://107.172.22.247/NST/video.mp4
Connecting to 107.172.22.247:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1570024 (1.5M) [video/mp4]
Saving to: ‘video.mp4’

video.mp4           100%[===================>]   1.50M  7.94MB/s    in 0.2s    

2020-11-30 22:02:09 (7.94 MB/s) - ‘video.mp4’ saved [1570024/1570024]



load the video for style transfer.

In [ ]:
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('video.mp4')

video = []

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    resized_frame = cv2.resize(frame, (256, 256))

    video.append(resized_frame)
    

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

video = np.array(video) / 255

Do the style transfer for the video.

In [ ]:
transfer_video = transform_model.predict(video)
transfer_video *= 255
transfer_video = transfer_video.astype(np.uint8)


Output the transfered video. 

In [ ]:

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4',fourcc, 30, (256,256)) # fps, (size)

for i in range(transfer_video.shape[0]):    
    frame = transfer_video[i]

    # write the flipped frame
    out.write(frame)


# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

